# 第七章 数据清洗和准备

In [2]:
import numpy as np
import pandas as pd

## 7.1 处理缺失数据

对于数值数据，pandas使用浮点值NaN (Not a Number) 表示缺失数据。
Python内置的None值在对象数组中也可以作为NA。
关于处理缺失值的函数有：
dropna  #根据各标签的值中是否存在缺失数据对轴标签进行过滤，可通过阈值调节对缺失值的容忍度
fillna  #用指定值或插值方法（如ffill或bfill）填充缺失数据
isnull  #返回一个含有布尔值的对象，这些布尔值表示哪些值是缺失值/NA,该对象的类型和源类型一样
notnull #isnull的否定式

### 滤除缺失数据

In [6]:
#对于一个Series，dropna返回一个仅含非空数据和索引值的Series:
from numpy import nan as NA
data= pd.Series ([1,NA, 3.5,NA,7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [7]:
#这等价于：
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

对于DataFrame对象较为复杂。dropna默认丢弃任何含有缺失值的行:

In [9]:
data=pd.DataFrame([[1.,6.5,3.],[1.,NA,NA],[NA,NA,NA],[NA,6.5,3.0]])
cleaned= data.dropna()
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [10]:
cleaned

,0,1,2
0,1.0,6.5,3.0


In [11]:
#传入how='all'将只丢弃全为NA的那些行
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [12]:
#用这种方式丢弃列，只需传入axis=1即可
data[4]=NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [13]:
data.dropna(axis=1,how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


另一个滤除DataFrame行的问题涉及时间序列数据，假设只想留下一部分观测数据，可以用thresh参数来实现：

In [14]:
df=pd.DataFrame(np.random.randn(7,3))
df.iloc[:4,1]=NA
df.iloc[:2,2]=NA
df

,0,1,2
0,-1.400361,NaN,NaN
1,0.330325,NaN,NaN
2,0.920915,NaN,-0.386213
3,-0.184638,NaN,0.062417
4,1.068463,-0.117446,0.954613
5,-0.153954,-2.208547,1.033770
6,0.127220,0.728118,1.296851


In [15]:
df.dropna()

,0,1,2
4,1.068463,-0.117446,0.954613
5,-0.153954,-2.208547,1.033770
6,0.127220,0.728118,1.296851


In [16]:
df.dropna(thresh=2)

,0,1,2
2,0.920915,NaN,-0.386213
3,-0.184638,NaN,0.062417
4,1.068463,-0.117446,0.954613
5,-0.153954,-2.208547,1.033770
6,0.127220,0.728118,1.296851


### 填充缺失数据

In [17]:
#最主要的函数为fillna
#通过一个常数调用fillna可以将缺失值替换成该常数值
df.fillna(0)

,0,1,2
0,-1.400361,0.000000,0.000000
1,0.330325,0.000000,0.000000
2,0.920915,0.000000,-0.386213
3,-0.184638,0.000000,0.062417
4,1.068463,-0.117446,0.954613
5,-0.153954,-2.208547,1.033770
6,0.127220,0.728118,1.296851


In [18]:
#若通过一个字典调用fillna，可以实现对不同的列填充不同的值
df.fillna({1:0.5,2:0})

,0,1,2
0,-1.400361,0.500000,0.000000
1,0.330325,0.500000,0.000000
2,0.920915,0.500000,-0.386213
3,-0.184638,0.500000,0.062417
4,1.068463,-0.117446,0.954613
5,-0.153954,-2.208547,1.033770
6,0.127220,0.728118,1.296851


In [19]:
#fillna默认会返回新对象，但也可以对现有对象进行就地修改
_=df.fillna(0,inplace=True)
df

,0,1,2
0,-1.400361,0.000000,0.000000
1,0.330325,0.000000,0.000000
2,0.920915,0.000000,-0.386213
3,-0.184638,0.000000,0.062417
4,1.068463,-0.117446,0.954613
5,-0.153954,-2.208547,1.033770
6,0.127220,0.728118,1.296851


In [20]:
#对reindexing有效的那些插值法也可用于fillna
df=pd.DataFrame(np.random.randn(6,3))
df.iloc[2:,1]=NA
df.iloc[4:,2]=NA
df

,0,1,2
0,-0.303035,-0.130312,0.303070
1,0.327503,-0.355268,0.215774
2,1.555385,NaN,1.171911
3,-1.009757,NaN,-0.513594
4,-0.625505,NaN,NaN
5,0.541895,NaN,NaN


In [21]:
df.fillna(method='ffill')

,0,1,2
0,-0.303035,-0.130312,0.303070
1,0.327503,-0.355268,0.215774
2,1.555385,-0.355268,1.171911
3,-1.009757,-0.355268,-0.513594
4,-0.625505,-0.355268,-0.513594
5,0.541895,-0.355268,-0.513594


In [22]:
df.fillna(method='ffill',limit=2)

,0,1,2
0,-0.303035,-0.130312,0.303070
1,0.327503,-0.355268,0.215774
2,1.555385,-0.355268,1.171911
3,-1.009757,-0.355268,-0.513594
4,-0.625505,NaN,-0.513594
5,0.541895,NaN,-0.513594


In [24]:
#其他功能，如传入Series的平均值或中位数
data=pd.Series([1.,NA,3.5,NA,7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

fillna函数参数：
value   #用于填充缺失值的标量值或字典对象
method  #插值方式，如果函数调用时未指定其他参数，默认为“ffill”
axis   #待填充的轴，默认axis=0
inplace #修改调用者对象而不产生副本
limit  #（对于前向和后向填充）可以连续填充的最大数量

## 7.2 数据转换 

### 移除重复数据

例子

In [26]:
data=pd.DataFrame({'k1':['one','two']*3+['two'],'k2':[1,1,2,3,3,4,4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


DataFrame的duplicated方法返回一个布尔型Series,表示各行是否是重复行

In [27]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [28]:
#drop_duplicates方法：返回DataFrame,重复的数组标为False
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


这两个方法默认判断全部列。
指定部分列进行重复项判断：假设还有一列值，且只希望根据k1列过滤重复项：

In [29]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


duplicated和drop_duplicates默认保留第一个出现的值组合
传入keep='last'保留最后一个：

In [30]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### 利用函数或映射进行数据数据转换

根据数组、Series或DataFrame列中的值来实现转换工作

In [2]:
import numpy as np
import pandas as pd

In [3]:
#有关肉类的数据
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [5]:
#添加一列表示该肉类食物来源的动物类型：编写一个不同肉类到动物的映射
meat_to_animal = { 'bacon': 'pig',  'pulled pork': 'pig', 'pastrami': 'cow',  'corned beef': 'cow',  'honey ham': 'pig',  'nova lox': 'salmon'}

In [6]:
#Series的map方法可以接受一个函数或含有映射关系的字典型对象，但这里有一个问题，即有些肉类的首字母大写了，而另一些没有
#使用Series的str.lower方法，将各个值转换成小写
lowercased = data['food'].str.lower()
lowercased
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [7]:
#或传入一个能够完成全部这些工作的函数
data['food'].map(lambda x: meat_to_animal[x.lower()])
#使用map是一种实现元素级转换以及其他数据清理⼯作的便捷⽅式

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### 替换值

利⽤fillna⽅法填充缺失数据可以看做值替换的⼀种特殊情况；
map可⽤于修改对象的数据⼦集；
replace则提供了⼀种实现该功能的更简单、更灵活的⽅式

In [8]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [9]:
#-999这个值可能是⼀个表示缺失数据的标记值。要将其替换为pandas能够理解的NA值，我们可以利⽤replace来产⽣⼀个新的Series（除⾮传⼊inplace=True）：
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [10]:
#⼀次性替换多个值，可以传⼊⼀个由待替换值组成的列表以及⼀个替换值：
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [11]:
#让每个值有不同的替换值：传递一个替换列表
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [12]:
#传入的参数可以为字典
data.replace({-999: np.nan, -1000: 0})
#data.replace⽅法与data.str.replace不同，后者做的是字符串的元素级替换

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### 重命名轴索引

跟Series中的值⼀样，轴标签也可以通过函数或映射进⾏转换，从⽽得到⼀个新的不同标签的对象。轴还可以被就地修改，⽽⽆
需新建⼀个数据结构。

In [13]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

In [14]:
#跟Series⼀样，轴索引也有⼀个map⽅法
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [15]:
#可以将其赋值给index，这样就可以对DataFrame进⾏就地修改
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [16]:
#创建数据集的转换版，而非修改原始数据，使用rename
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [17]:
#rename可以结合字典型对象实现对部分轴标签的更新
data.rename(index={'OHIO':'INDIANA'},columns={'three':'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [18]:
#rename可以实现复制DataFrame并对其索引和列标签进⾏赋值。如果希望就地修改某个数据集，传⼊inplace=True即可
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### 离散化和面元划分

为了便于分析，连续数据常常被离散化或拆分为“⾯元”（bin）

In [19]:
#假设有⼀组⼈员数据
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [20]:
bins = [18, 25, 35, 60, 100]  #将数据划分成“18到25”、 “26到35”、 “35到60”以及“60以上”⼏个⾯元
cats = pd.cut(ages, bins)     #使用pandas的cut函数
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

pandas返回的是⼀个特殊的Categorical对象。结果展示了pandas.cut划分的⾯元。
可以将其看做⼀组表示⾯元名称的字符串。
它的底层含有⼀个表示不同分类名称的类型数组，以及⼀个codes属性中的年龄数据的标签：

In [22]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [23]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

In [24]:
pd.value_counts(cats)  #pandas.cut结果的面元计数；

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [25]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False) #开闭端可以通过right=False进行修改

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [26]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior'] #设置面元名称
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

如果向cut传⼊的是⾯元的数量⽽不是确切的⾯元边界，则它会根据数据的最⼩值和最⼤值计算等⻓⾯元

In [27]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2) #将⼀些均匀分布的数据分成四组；选项precision=2，限定⼩数只有两位

[(0.71, 0.94], (0.26, 0.48], (0.71, 0.94], (0.26, 0.48], (0.71, 0.94], ..., (0.26, 0.48], (0.48, 0.71], (0.03, 0.26], (0.03, 0.26], (0.26, 0.48]]
Length: 20
Categories (4, interval[float64]): [(0.03, 0.26] < (0.26, 0.48] < (0.48, 0.71] < (0.71, 0.94]]

qcut是⼀个⾮常类似于cut的函数，它可以根据样本分位数对数据进⾏⾯元划分。根据数据的分布情况， cut可能⽆法使各个⾯元
中含有相同数量的数据点。⽽qcut由于使⽤的是样本分位数，因此可以得到⼤⼩基本相等的⾯元：

In [28]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)  # Cut into quartiles
cats

[(-0.662, -0.0361], (-0.0361, 0.643], (0.643, 3.145], (0.643, 3.145], (-3.072, -0.662], ..., (0.643, 3.145], (0.643, 3.145], (-0.662, -0.0361], (-0.662, -0.0361], (0.643, 3.145]]
Length: 1000
Categories (4, interval[float64]): [(-3.072, -0.662] < (-0.662, -0.0361] < (-0.0361, 0.643] < (0.643, 3.145]]

In [29]:
pd.value_counts(cats)

(0.643, 3.145]       250
(-0.0361, 0.643]     250
(-0.662, -0.0361]    250
(-3.072, -0.662]     250
dtype: int64

In [30]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]) #与cut类似，可以传递⾃定义的分位数（0到1之间的数值，包含端点）

[(-1.243, -0.0361], (-0.0361, 1.261], (1.261, 3.145], (1.261, 3.145], (-1.243, -0.0361], ..., (-0.0361, 1.261], (-0.0361, 1.261], (-1.243, -0.0361], (-1.243, -0.0361], (1.261, 3.145]]
Length: 1000
Categories (4, interval[float64]): [(-3.072, -1.243] < (-1.243, -0.0361] < (-0.0361, 1.261] < (1.261, 3.145]]

聚合和分组运算时会再次⽤到cut和qcut，因为这两个离散化函数对分位和分组分析⾮常重要

### 检测和过滤异常值

过滤或变换异常值（outlier）在很⼤程度上就是运⽤数组运算

In [31]:
#一个含有正态分布数据的DataFrame
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.008413,-0.060735,0.016590,0.023117
std,0.996513,0.994514,1.009953,0.997778
min,-3.151843,-3.088770,-3.592982,-2.607545
25%,-0.630914,-0.682075,-0.652801,-0.674259
50%,0.014378,-0.084665,0.009674,0.018765
75%,0.663042,0.562097,0.708803,0.699818
max,3.115308,2.909542,3.210786,3.080497


In [32]:
#找出某列中绝对值⼤⼩超过3的值
col = data[2]
col[np.abs(col) > 3]

15    -3.179568
482    3.210786
494   -3.592982
896   -3.256002
Name: 2, dtype: float64

In [33]:
data[(np.abs(data) > 3).any(1)] #要选出全部含有“超过3或－ 3的值”的⾏，可以在布尔型DataFrame中使⽤any⽅法

,0,1,2,3
15,-0.052223,-1.405696,-3.179568,-0.388764
116,3.001485,-0.479873,-0.333809,0.180572
137,0.091973,0.061110,1.205795,3.080497
231,-3.140135,-0.754863,-1.295099,-1.643821
284,3.101668,0.446944,-0.805558,1.484042
482,0.716475,0.898295,3.210786,0.325521
484,3.115308,1.654426,-0.894594,-1.696723
494,-0.906928,0.661282,-3.592982,0.593747
642,-3.151843,0.015565,1.000229,0.772719
682,-3.093379,-0.184238,-0.782097,0.379292


In [34]:
data[np.abs(data) > 3] = np.sign(data) * 3 #将值限制在区间-3到3以内
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.008580,-0.060647,0.017407,0.022997
std,0.994652,0.994248,1.006002,0.997415
min,-3.000000,-3.000000,-3.000000,-2.607545
25%,-0.630914,-0.682075,-0.652801,-0.674259
50%,0.014378,-0.084665,0.009674,0.018765
75%,0.663042,0.562097,0.708803,0.699818
max,3.000000,2.909542,3.000000,3.000000


In [35]:
np.sign(data).head()  #根据数据的值是正还是负， np.sign(data)可以⽣成1和-1

,0,1,2,3
0,-1.0,-1.0,-1.0,-1.0
1,1.0,-1.0,-1.0,1.0
2,-1.0,1.0,-1.0,1.0
3,-1.0,-1.0,1.0,1.0
4,-1.0,-1.0,1.0,1.0


### 排列和随机采样

利⽤numpy.random.permutation函数可以实现对Series或DataFrame的列的排列⼯作（permuting，随机重排序）。
通过需要排列的轴的⻓度调⽤permutation，可产⽣⼀个表示新顺序的整数数组：

In [36]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

array([3, 4, 2, 0, 1])

In [37]:
#然后就可以在基于iloc的索引操作或take函数中使⽤该数组
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [38]:
df.take(sampler)

,0,1,2,3
3,12,13,14,15
4,16,17,18,19
2,8,9,10,11
0,0,1,2,3
1,4,5,6,7


In [39]:
df.sample(n=3)  #如果不想⽤替换的⽅式选取随机⼦集，可以在Series和DataFrame上使⽤sample⽅法

,0,1,2,3
1,4,5,6,7
3,12,13,14,15
2,8,9,10,11


In [40]:
choices = pd.Series([5, 7, -1, 6, 4]) #要通过替换的⽅式产⽣样本（允许重复选择），可以传递replace=True到sample
draws = choices.sample(n=10, replace=True)
draws

4    4
3    6
4    4
2   -1
2   -1
1    7
2   -1
4    4
4    4
3    6
dtype: int64

### 计算指标/哑变量

将分类变量（categorical variable）转换为“哑变量”或“指标矩阵”
如果DataFrame的某⼀列中含有k个不同的值，则可以派⽣出⼀280个k列矩阵或DataFrame（其值全为1和0）
pandas有⼀个get_dummies函数可以实现该功能

In [41]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)})  #使用之前的例子
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [42]:
dummies = pd.get_dummies(df['key'], prefix='key')  #给指标DataFrame的列加上⼀个前缀，以便能够跟其他数据进⾏合并
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


如果DataFrame中的某⾏同属于多个分类，则事情就会有点复杂。看⼀下MovieLens 1M数据集， 14章会更深⼊地研究它

In [ ]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames)
movies[:10]

要为每个genre添加指标变量就需要做⼀些数据规整操作；
首先，从数据集中抽取出不同的genre值：

In [ ]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)

构建指标DataFrame的⽅法之⼀是从⼀个全零DataFrame开始：

In [ ]:
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)

现在，迭代每⼀部电影，并将dummies各⾏的条⽬设为1

In [ ]:
#使⽤dummies.columns来计算每个类型的列索引
gen = movies.genres[0]
gen.split('|')
dummies.columns.get_indexer(gen.split('|'))

In [ ]:
#根据索引，使用.iloc设定值
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [ ]:
#与movies合并起来
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]
#对于很⼤的数据，⽤这种⽅式构建多成员指标变量就会变得⾮常慢。最好使⽤更低级的函数，将其写⼊NumPy数组，然后结果包装在DataFrame中。

In [44]:
#⼀个对统计应⽤有⽤的秘诀是：结合get_dummies和诸如cut之类的离散化函数：
np.random.seed(12345)
values = np.random.rand(10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [45]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## 7.3 字符串操作

### 字符串对象方法

In [46]:
val = 'a,b,  guido'  #以逗号分隔的字符串可以⽤split拆分成数段
val.split(',')

['a', 'b', '  guido']

In [47]:
pieces = [x.strip() for x in val.split(',')] #split常常与strip⼀起使⽤，以去除空⽩符（包括换⾏符）
pieces

['a', 'b', 'guido']

In [48]:
first, second, third = pieces  #利⽤加法，可以将这些⼦字符串以双冒号分隔符的形式连接起来
first + '::' + second + '::' + third 

'a::b::guido'

In [49]:
#一种更快的方式：向字符串"::"的join方法传入一个列表或元组
'::'.join(pieces)

'a::b::guido'

其它⽅法关注的是⼦串定位。检测⼦串的最佳⽅式是利⽤Python的in关键字，还可以使⽤index和find

In [50]:
'guido' in val

True

In [51]:
val.index(',')

1

In [52]:
val.find(':')  

-1

In [53]:
#注意find和index的区别：如果找不到字符串， index将会引发⼀个异常（⽽不是返回－ 1）
val.index(':')

ValueError: substring not found

In [54]:
#count可以返回指定⼦串的出现次数：
val.count(',')

2

In [55]:
#replace⽤于将指定模式替换为另⼀个模式。通过传⼊空字符串，它也常常⽤于删除模式：
val.replace(',', '::')

'a::b::  guido'

In [56]:
val.replace(',', '')

'ab  guido'

这些运算大部分都可以使用正则表达式实现